In [28]:
#%pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks sentence-transformers


In [29]:
api_key="57063513-3520-41ff-bbbc-c750bc7d9b49"

In [30]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [31]:
import os 
from pinecone import Pinecone, ServerlessSpec

index_name="hybrid-search-langchain-pinecone"

# Initialize PineCone client
pc=Pinecone(api_key=api_key)


#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, 
        metric='dotproduct', ## sparse values supported for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )


In [32]:
index=pc.Index(index_name)
index

In [33]:
# vector embedding and sparse matrix
import os

from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
load_dotenv()

# from langchain_huggingface import HuggingFaceEmbeddings
# embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# embeddings

goog_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
goog_embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7b354180a610>, model='models/text-embedding-004', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None)

In [41]:
from pinecone_text.sparse import BM25Encoder # uses tfidf technique by default

bm25_encoder=BM25Encoder()
bm25_encoder

In [44]:
# corpus=[
#     "In 2023, I visited Paris",
#     "In 2022, I visited New York",
#     "In 2021, I visited New Orleans"]
corpus = ["The quick brown fox jumps over the lazy dog",
          "The lazy dog is brown",
          "The fox is brown"]

## tfid values on these sentences
bm25_encoder.fit(corpus)

## store the values to a json file
# bm25_encoder.dump("bm25_values.json")



  0%|          | 0/3 [00:00<?, ?it/s]

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/mandy/nltk_data'
    - '/home/mandy/dev/pyenv/nltk_data'
    - '/home/mandy/dev/pyenv/share/nltk_data'
    - '/home/mandy/dev/pyenv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [36]:
retriever=PineconeHybridSearchRetriever(embeddings=goog_embeddings, sparse_encoder=bm25_encoder, index=index)

In [39]:
retriever.add_texts([
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
])

  0%|          | 0/1 [00:00<?, ?it/s]

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/mandy/nltk_data'
    - '/home/mandy/dev/pyenv/nltk_data'
    - '/home/mandy/dev/pyenv/share/nltk_data'
    - '/home/mandy/dev/pyenv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
